In [1]:
#@markdown #**Exploring DQD Algorithms** 🔬

In [2]:
#@markdown #**Check GPU type** 🕵️
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good

#@markdown T4 = Good (*preferred*)

#@markdown K80 = Meh

#@markdown P4 = (*Not Recommended*) 

#@markdown ---

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5a113c18-ce54-64b9-f13d-14d3e5c8b89d)


In [3]:
#@markdown #**Install libraries** 🏗️
# @markdown This cell will take around 4 minutes to download several libraries.

#@markdown ---
!pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!git clone https://github.com/openai/CLIP
!pip install -e ./CLIP
!pip install einops ninja
!curl -LO 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-1024x1024.pkl'

!git clone https://github.com/icaros-usc/pyribs.git
!pip install ./pyribs[all]

import sys
sys.path.append("./CLIP")
sys.path.append("./stylegan2-ada-pytorch")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:42tcmalloc: large alloc 1147494400 bytes == 0x3a7f4000 @  0x7fcde9f5d615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:47tcmalloc: large alloc 1434370048 bytes == 0x7ee4a000 @  0x7fcde9f5d615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [4]:
import os
import csv
import time
from pathlib import Path

import clip

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from einops import rearrange

import pickle
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook, tqdm
from PIL import Image

from ribs.archives import CVTArchive, GridArchive
from ribs.emitters import GradientAborescenceEmitter
from ribs.schedulers import Scheduler
from ribs.visualize import grid_archive_heatmap


In [5]:
def tensor_to_pil_img(img):
    img = (img.clamp(-1, 1) + 1) / 2.0
    img = img[0].permute(1, 2, 0).detach().cpu().numpy() * 255
    img = Image.fromarray(img.astype('uint8'))
    return img

def norm1(prompt):
    return prompt / prompt.square().sum(dim=-1, keepdim=True).sqrt()

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)

def cos_sim_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().mul(2)

def prompts_dist_loss(x, targets, loss):
    distances = [loss(x, target) for target in targets]
    return torch.stack(distances, dim=-1).sum(dim=-1)

In [6]:
class MakeCutouts(torch.nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.0):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

    def forward(self, x):
        sideY, sideX = x.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([]) ** self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = x[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
        return torch.cat(cutouts)

make_cutouts = MakeCutouts(224, 32, 0.5)

In [7]:
class CLIP(object):
    def __init__(self, device='cpu'):
        self.device = device
        clip_model_name = "ViT-B/32"
        self.model, _ = clip.load(clip_model_name, device=device)
        self.model = self.model.requires_grad_(False)
        self.model.eval()
        self.normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                              std=[0.26862954, 0.26130258, 0.27577711])

    @torch.no_grad()
    def embed_text(self, prompt):
        return norm1(self.model.encode_text(clip.tokenize(prompt)
               .to(self.device)).float())

    def embed_cutout(self, image):
        return norm1(self.model.encode_image(self.normalize(image)))

    def embed_image(self, image):
        n = image.shape[0]
        cutouts = make_cutouts(image)
        embeds = self.embed_cutout(cutouts)
        embeds = rearrange(embeds, '(cc n) c -> cc n c', n=n)
        return embeds

In [8]:
class Generator(object):

    def __init__(self, device='cpu'):
        self.device = device
        model_filename = 'stylegan2-ffhq-1024x1024.pkl'
        with open(model_filename, 'rb') as fp:
            self.model = pickle.load(fp)['G_ema'].to(device)
            self.model.eval()
        for p in self.model.parameters():
            p.requires_grad_(False)
        self.init_stats()
        self.latent_shape = (-1, 512)

    def init_stats(self):
        zs = torch.randn([10000, self.model.mapping.z_dim], device=self.device)
        ws = self.model.mapping(zs, None)
        self.w_stds = ws.std(0)
        qs = ((ws - self.model.mapping.w_avg) / self.w_stds).reshape(10000, -1)
        self.q_norm = torch.norm(qs, dim=1).mean() * 0.15

    def gen_random_ws(self, num_latents):
        zs = torch.randn([num_latents, self.model.mapping.z_dim], device=self.device)
        ws = self.model.mapping(zs, None)
        return ws

In [9]:
class Classifier(object):

    def __init__(self, gen_model, class_model, celebrity_id='Musk'):
        self.device = gen_model.device
        self.gen_model = gen_model
        self.class_model = class_model
        self.measures = []
        if celebrity_id == 'Beyonce':
            self.init_objective('A photo of the face of Beyonce.')
            self.add_measure('A photo of Beyonce as a small child.', 
                             'A photo of Beyonce as an elderly person.')
            self.add_measure('A photo of Beyonce with short hair.', 
                             'A photo of Beyonce with long hair.')
        elif celebrity_id == 'Cruise':
            self.init_objective('A photo of the face of Tom Cruise.')
            self.add_measure('A photo of Tom Cruise as a small child.', 
                             'A photo of Tom Cruise as an elderly person.')
            self.add_measure('A photo of Tom Cruise with short hair.', 
                             'A photo of Tom Cruise with long hair.')
        elif celebrity_id == 'Lopez':
            self.init_objective('A photo of the face of Jennifer Lopez.')
            self.add_measure('A photo of Jennifer Lopez as a small child.', 
                             'A photo of Jennifer Lopez as an elderly person.')
            self.add_measure('A photo of Jennifer Lopez with short hair.', 
                             'A photo of Jennifer Lopez with long hair.')
        elif celebrity_id == 'Musk':
            self.init_objective('A photo of the face of Elon Musk.')
            self.add_measure('A photo of Elon Musk as a small child.', 
                             'A photo of Elon Musk as an elderly person.')
            self.add_measure('A photo of Elon Musk with short hair.', 
                             'A photo of Elon Musk with long hair.')
        else:
            print('The celebrity \"{}\" is not a valid option.'.format(celebrity_id))
            exit(0)

    def init_objective(self, text_prompt):
        texts = [frase.strip() for frase in text_prompt.split("|") if frase]
        self.obj_targets = [self.class_model.embed_text(text) for text in texts]

    def add_measure(self, positive_text, negative_text):
        texts = [frase.strip() for frase in positive_text.split("|") if frase]
        negative_targets = [self.class_model.embed_text(text) for text in texts]
        
        texts = [frase.strip() for frase in negative_text.split("|") if frase]
        positive_targets = [self.class_model.embed_text(text) for text in texts]
        
        self.measures.append((negative_targets, positive_targets))

    def find_good_start_latent(self, batch_size=16, num_batches=32):
        with torch.inference_mode():
            qs = []
            losses = []
            G = self.gen_model.model
            w_stds = self.gen_model.w_stds
            for _ in range(num_batches):
                q = (G.mapping(torch.randn([batch_size, G.mapping.z_dim], device=self.device),
                    None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds
                images = G.synthesis(q * w_stds + G.mapping.w_avg)
                embeds = self.class_model.embed_image(images.add(1).div(2))
                loss = prompts_dist_loss(embeds, self.obj_targets, spherical_dist_loss).mean(0)
                i = torch.argmin(loss)
                qs.append(q[i])
                losses.append(loss[i])
            qs = torch.stack(qs)
            losses = torch.stack(losses)

            i = torch.argmin(losses)
            q = qs[i].unsqueeze(0)

        return q.flatten()

    def generate_image(self, latent_code):
        ws, _ = self.transform_to_w([latent_code])
        images = self.gen_model.model.synthesis(ws, noise_mode='const')
        return images

    def transform_to_w(self, latent_codes):
        qs = []
        ws = []
        for cur_code in latent_codes:
            q = torch.tensor(
                    cur_code.reshape(self.gen_model.latent_shape), 
                    device=self.device,
                    requires_grad=True,
                )
            qs.append(q)
            w = q * self.gen_model.w_stds + self.gen_model.model.mapping.w_avg
            ws.append(w)

        ws = torch.stack(ws, dim=0)
        return ws, qs

    def compute_objective_loss(self, embeds, qs, dim=None):
        loss = prompts_dist_loss(embeds, self.obj_targets, spherical_dist_loss).mean(0)

        diff = torch.max(torch.norm(qs, dim=dim), self.gen_model.q_norm)
        reg_loss = (diff - self.gen_model.q_norm).pow(2)
        loss = loss + 0.1 * reg_loss

        return loss

    def compute_objective(self, sols):
        ws, qs = self.transform_to_w(sols)

        images = self.gen_model.model.synthesis(ws, noise_mode='const')
        embeds = self.class_model.embed_image(images.add(1).div(2))
    
        loss = self.compute_objective_loss(embeds, qs[0])
        loss.backward()

        value = loss.cpu().detach().numpy()
        jacobian = -qs[0].grad.cpu().detach().numpy()
        return value, jacobian.flatten()

    def compute_measure(self, index, sols):
        ws, qs = self.transform_to_w(sols)

        images = self.gen_model.model.synthesis(ws, noise_mode='const')
        embeds = self.class_model.embed_image(images.add(1).div(2))

        measure_targets = self.measures[index]
        pos_loss = prompts_dist_loss(embeds, measure_targets[0], cos_sim_loss).mean(0)
        neg_loss = prompts_dist_loss(embeds, measure_targets[1], cos_sim_loss).mean(0)
        loss = pos_loss - neg_loss
        loss.backward()

        value = loss.cpu().detach().numpy()
        jacobian = qs[0].grad.cpu().detach().numpy()
        return value, jacobian.flatten()

    def compute_measures(self, sols):
    
        values = []
        jacobian = []
        for i in range(len(self.measures)):
            value, jac = self.compute_measure(i, sols)
            values.append(value)
            jacobian.append(jac)

        return np.stack(values, axis=0), np.stack(jacobian, axis=0)

    def compute_all(self, sols):
        with torch.inference_mode():

            ws, qs = self.transform_to_w(sols)
            qs = torch.stack(qs, dim=0)

            images = self.gen_model.model.synthesis(ws, noise_mode='const')
            embeds = self.class_model.embed_image(images.add(1).div(2))
            
            values = []
            loss = self.compute_objective_loss(embeds, qs, dim=(1,2))
            value = loss.cpu().detach().numpy()
            values.append(value)

            for i in range(len(self.measures)):
                measure_targets = self.measures[i]
                pos_loss = prompts_dist_loss(
                        embeds, 
                        measure_targets[0], 
                        cos_sim_loss,
                    ).mean(0)
                neg_loss = prompts_dist_loss(
                        embeds, 
                        measure_targets[1], 
                        cos_sim_loss
                    ).mean(0)
                loss = pos_loss - neg_loss
                value = loss.cpu().detach().numpy()
                values.append(value)

        return np.stack(values, axis=0)

In [10]:
def transform_obj(objs):
    # Remap the objective from minimizing [0, 20] to maximizing [0, 100]
    return (20.0-objs)*5.0

def create_optimizer(algorithm, classifier, seed):
    """Creates an optimizer based on the algorithm name.

    Args:
        algorithm (str): Name of the algorithm passed into sphere_main.
        classifier (Classifier): The models for the search.
        seed (int): Main seed or the various components.
    Returns:
        Scheduler: A ribs Scheduler for running the algorithm.
    """
    bounds = [(-0.3, 0.3), (-0.3, 0.3)]
    initial_sol = classifier.find_good_start_latent().cpu().detach().numpy()
    dim = len(initial_sol)
    batch_size = 36
    num_emitters = 1
    resolution = 200
    grid_dims = (resolution, resolution)

    # Create archive.
    archive = GridArchive(
        solution_dim = dim,
        dims=grid_dims, 
        ranges=bounds, 
        learning_rate=0.02,
        threshold_min=0.0,
        seed=seed
    )

    # Maintain a passive elitist archive
    passive_archive = GridArchive(
        solution_dim = dim,
        dims=grid_dims,
        ranges=bounds,
        seed=seed
    )

    # Create emitters. Each emitter needs a different seed, so that they do not
    # all do the same thing.
    emitter_seeds = [None] * num_emitters if seed is None else list(
        range(seed, seed + num_emitters))
    emitters = [
        GradientAborescenceEmitter(
            archive=archive,
            x0=initial_sol,
            sigma0=10.0,
            step_size=0.03,
            restart_rule='basic',
            batch_size=batch_size,
            seed=s
        ) for s in emitter_seeds
    ]

    return Scheduler(archive, emitters), passive_archive

In [11]:
def save_heatmap(archive, heatmap_path):
    """Saves a heatmap of the archive to the given path.
    Args:
        archive (GridArchive or CVTArchive): The archive to save.
        heatmap_path: Image path for the heatmap.
    """
    plt.figure(figsize=(8, 6))
    grid_archive_heatmap(archive, vmin=0, vmax=100, cmap="viridis")
    plt.tight_layout()
    plt.savefig(heatmap_path)
    plt.close(plt.gcf())

In [12]:
def sphere(solution_batch):
    """Sphere function evaluation and measures for a batch of solutions.

    Args:
        solution_batch (np.ndarray): (batch_size, dim) batch of solutions.
    Returns:
        objective_batch (np.ndarray): (batch_size,) batch of objectives.
        measures_batch (np.ndarray): (batch_size, 2) batch of measures.
    """
    dim = solution_batch.shape[1]

    # Shift the Sphere function so that the optimal value is at x_i = 2.048.
    sphere_shift = 5.12 * 0.4

    # Normalize the objective to the range [0, 100] where 100 is optimal.
    best_obj = 0.0
    worst_obj = (-5.12 - sphere_shift)**2 * dim
    raw_obj = np.sum(np.square(solution_batch - sphere_shift), axis=1)
    objective_batch = (raw_obj - worst_obj) / (best_obj - worst_obj) * 100

    # Compute gradient of the objective
    objective_grad_batch = -2 * (solution_batch - sphere_shift)

    # Calculate measures.
    clipped = solution_batch.copy()
    clip_indices = np.where(np.logical_or(clipped > 5.12, clipped < -5.12))
    clipped[clip_indices] = 5.12 / clipped[clip_indices]
    measures_batch = np.concatenate(
        (
            np.sum(clipped[:, :dim // 2], axis=1, keepdims=True),
            np.sum(clipped[:, dim // 2:], axis=1, keepdims=True),
        ),
        axis=1,
    )

    # Compute gradient of the measures
    derivatives = np.ones(solution_batch.shape)
    derivatives[clip_indices] = -5.12 / np.square(solution_batch[clip_indices])

    mask_0 = np.concatenate((np.ones(dim // 2), np.zeros(dim - dim // 2)))
    mask_1 = np.concatenate((np.zeros(dim // 2), np.ones(dim - dim // 2)))

    d_measure0 = derivatives * mask_0
    d_measure1 = derivatives * mask_1

    measures_grad_batch = np.stack((d_measure0, d_measure1), axis=1)

    return (
        objective_batch,
        objective_grad_batch,
        measures_batch,
        measures_grad_batch,
    )

In [13]:
# Initialization.
celebrity='Musk'
outdir='logs'
            
# Create a shared logging directory for the experiments for this algorithm.
s_logdir = os.path.join(outdir, "cma_mae")
logdir = Path(s_logdir)
outdir = Path(outdir)
if not outdir.is_dir():
    outdir.mkdir()
if not logdir.is_dir():
    logdir.mkdir()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

clip_model = CLIP(device=device)
gen_model = Generator(device=device)
classifier = Classifier(gen_model, clip_model, celebrity_id=celebrity)

100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 47.6MiB/s]


Setting up PyTorch plugin "bias_act_plugin"... Done.


In [14]:
#StyleGAN+CLIP LSI experiments.
trials=2
init_pop=100
itrs=1000
log_freq=1
log_arch_freq=1000
image_monitor=False
image_monitor_freq=5
seed=None

for trial_id in range(trials):
    print("Running trial", trial_id)
    # Create a directory for this specific trial.
    s_logdir = os.path.join(outdir, "cma_mae", f"trial_{trial_id}")
    logdir = Path(s_logdir)
    if not logdir.is_dir():
        logdir.mkdir()

    # Create a directory for logging intermediate images if the monitor is on.
    if image_monitor:
        image_monitor_freq = max(1, image_monitor_freq)
        gen_output_dir = os.path.join('generations')
        logdir = Path(gen_output_dir)
        if not logdir.is_dir():
            logdir.mkdir()
        gen_output_dir = os.path.join('generations', f"trial_{trial_id}")
        logdir = Path(gen_output_dir)
        if not logdir.is_dir():
            logdir.mkdir()

    # Create a new summary file
    summary_filename = os.path.join(s_logdir, "summary.csv")
    if os.path.exists(summary_filename):
        os.remove(summary_filename)
    with open(summary_filename, 'w') as summary_file:
        writer = csv.writer(summary_file)
        writer.writerow(['Iteration', 'QD-Score', 'Coverage', 'Maximum', 'Average'])
    
    scheduler, passive_archive = create_optimizer(algorithm="cma_mae", classifier=classifier,seed = seed)
    archive = scheduler.archive

    best = -1000
    non_logging_time = 0.0
    for itr in tqdm(range(1, itrs + 1)):
        itr_start = time.time()
        
        solution_batch = scheduler.ask_dqd()
        (objective_batch, objective_grad_batch, measures_batch,
         measures_grad_batch) = sphere(solution_batch)
        objective_grad_batch = np.expand_dims(objective_grad_batch, axis=1)
        jacobian_batch = np.concatenate(
                (objective_grad_batch, measures_grad_batch), axis=1)
        scheduler.tell_dqd(objective_batch, measures_batch, jacobian_batch)

        sols = scheduler.ask()
        values = classifier.compute_all(sols)
        values = np.transpose(values)

        objs = values[:,0]
        measures = values[:,1:3]

        objs = transform_obj(np.array(objs, dtype=np.float32))
        measures = np.array(measures, dtype=np.float32)
        best_gen = max(objs) 
        best = max(best, best_gen)

        scheduler.tell(objs, measures)

        non_logging_time += time.time() - itr_start

        if itr%50 == 0:
          print('best', best, best_gen)

        if image_monitor and itr % image_monitor_freq == 0:
            best_index = np.argmax(objs)
            latent_code = sols[best_index]

            img = classifier.generate_image(latent_code)
            img = tensor_to_pil_img(img)
            img.save(os.path.join(gen_output_dir, f'{itr}.png'))

        # Save the archive at the given frequency.
        # Always save on the final iteration.
        final_itr = itr == itrs
        if (itr > 0 and itr % log_arch_freq == 0) or final_itr:
            # Save a full archive for analysis.
            df = passive_archive.as_pandas(include_solutions = final_itr)
            df.to_pickle(os.path.join(s_logdir, f"archive_{itr:08d}.pkl"))

            # Save a heatmap image to observe how the trial is doing.
            save_heatmap(passive_archive, os.path.join(s_logdir, f"heatmap_{itr:08d}.png"))

        # Update the summary statistics for the archive
        if (itr > 0 and itr % log_freq == 0) or final_itr:
            with open(summary_filename, 'a') as summary_file:
                writer = csv.writer(summary_file)

                sum_obj = 0
                num_filled = 0
                num_bins = passive_archive._cells
                for obj in passive_archive._objective_arr:
                    num_filled += 1
                    sum_obj += obj
                qd_score = sum_obj / num_bins
                average = sum_obj / num_filled
                coverage = 100.0 * num_filled / num_bins
                data = [itr, qd_score, coverage, best, average]
                writer.writerow(data)

Running trial 0
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


  0%|          | 0/1000 [00:00<?, ?it/s]

best 95.500275 95.49053
best 95.501175 95.491745
best 95.51198 95.50697
best 95.545845 94.85664
best 95.56583 95.43601
best 95.58491 95.483025
best 95.58491 95.468605
best 95.64187 40.236244
best 95.64187 95.457985
best 95.64187 95.63048
best 95.64358 95.5954
best 95.64358 95.50144
best 95.64358 95.492905
best 95.64358 95.467865
best 95.64358 95.46205
best 95.64358 95.42095
best 95.64358 95.4295
best 95.64358 95.44711
best 95.64358 95.4306
best 95.64358 95.487976
Running trial 1


  0%|          | 0/1000 [00:00<?, ?it/s]

best 95.475075 95.4129
best 95.52494 95.50602
best 95.52494 95.46629
best 95.52494 90.41577
best 95.52494 95.44537
best 95.52494 95.46062
best 95.52494 91.473


KeyboardInterrupt: ignored